In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
df = pd.read_csv('./Data/EHB_ACTIVE_GRANT_FA_AGR.csv')

In [3]:
# df.head()
# df.columns

In [3]:
# only keep relevant columns
grant_data = df[['Award Year', 'Total Active Grant Financial Assistance', 'Grantee City', 'State Name', 'Grantee ZIP Code', 'Uniform Data System Grant Program Description', 'Grantee County Name', 'Complete County Name','HRSA Region']]

In [5]:
grant_data.tail()

,Award Year,Total Active Grant Financial Assistance,Grantee City,State Name,Grantee ZIP Code,Uniform Data System Grant Program Description,Grantee County Name,Complete County Name,HRSA Region
24213,2019,725000.0,Morehead,Kentucky,40351-1179,The Rural Communities Opioid Response Program ...,Rowan,Rowan County,Region 4
24214,2021,99058.0,Frackville,Pennsylvania,17931-1719,This notice announces the opportunity to apply...,Schuylkill,Schuylkill County,Region 3
24215,2021,500000.0,Tucson,Arizona,85704-3850,The Rural Communities Opioid Response Program ...,Pima,Pima County,Region 9
24216,2021,49529.0,Spearman,Texas,79081-3441,This notice announces the opportunity to apply...,Hansford,Hansford County,Region 6
24217,2021,9301536.0,Lincoln,Nebraska,68508-2529,?The program will provide increased COVID-19 t...,Lancaster,Lancaster County,Region 7


In [4]:
#filter for only awards in 2019 
grant_data_2015_2019 = grant_data[(grant_data['Award Year'] >= 2015) & (grant_data['Award Year'] <= 2019)]
grant_data_2015_2019 = grant_data_2015_2019.reset_index()

#dropna values 
grant_data_2015_2019 = grant_data_2015_2019.dropna()

In [ ]:
# now we want to get county level data
# go from county to population size and determine funding for large vs small counties 

In [5]:
cbsa = pd.read_csv('./Data/cbsa2fipsxw.csv')
cbsa.head()

,cbsacode,metrodivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33860.0,NaN,NaN,"Montgomery, AL",Metropolitan Statistical Area,NaN,NaN,Autauga County,Alabama,1.0,1.0,Central
2,19300.0,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1.0,3.0,Central
3,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Bibb County,Alabama,1.0,7.0,Outlying
4,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Blount County,Alabama,1.0,9.0,Outlying


In [8]:
grant_data_2015_2019.head()

,index,Award Year,Total Active Grant Financial Assistance,Grantee City,State Name,Grantee ZIP Code,Uniform Data System Grant Program Description,Grantee County Name,Complete County Name,HRSA Region
3,28,2019,1000000.0,Asheville,North Carolina,28801-4136,The Rural Communities Opioid Response Program ...,Buncombe,Buncombe County,Region 4
4,34,2019,1000000.0,Dixon,Illinois,61021-3116,The Rural Communities Opioid Response Program ...,Lee,Lee County,Region 5
10,46,2019,281553.0,Tulsa,Oklahoma,74107-1886,This notice announces the opportunity to apply...,Tulsa,Tulsa County,Region 6
12,60,2018,199849.0,Albia,Iowa,52531-8793,This notice announces the opportunity to apply...,Monroe,Monroe County,Region 7
13,61,2019,199503.0,Albia,Iowa,52531-8793,This notice announces the opportunity to apply...,Monroe,Monroe County,Region 7


In [6]:
# get grant assistance per state 
# grants_per_county = grant_data_2019.groupby(by='Complete County Name')['Total Active Grant Financial Assistance'].sum().reset_index()
# grants_per_county.head()

grant_data_2015_2019['Mental Health Assistance'] = grant_data_2015_2019.apply(lambda row: row['Total Active Grant Financial Assistance']\
                                                                               if 'mental health' in row['Uniform Data System Grant Program Description'] else 0, axis =1)

grants_per_county = grant_data_2015_2019.groupby(by=['Complete County Name', 'State Name','Award Year'])[['Total Active Grant Financial Assistance', 'Mental Health Assistance']].sum().reset_index()
grants_per_county.head()



,Complete County Name,State Name,Award Year,Total Active Grant Financial Assistance,Mental Health Assistance
0,Accomack County,Virginia,2017,52817.0,0.0
1,Accomack County,Virginia,2018,4973949.0,0.0
2,Accomack County,Virginia,2019,5397224.0,0.0
3,Ada County,Idaho,2017,1560323.0,10000.0
4,Ada County,Idaho,2018,3254149.0,225000.0


In [7]:
cbsa_data = cbsa[['countycountyequivalent', 'centraloutlyingcounty', 'statename', 'cbsacode','metropolitanmicropolitanstatis']]

#cbsa_data.head()
# merge total grants and mental health grants
grants_per_county_cbsa = grants_per_county.merge(cbsa_data, how='left', left_on=['Complete County Name','State Name'], right_on=['countycountyequivalent','statename'])

grants_per_county_cbsa.head()

,Complete County Name,State Name,Award Year,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,metropolitanmicropolitanstatis
0,Accomack County,Virginia,2017,52817.0,0.0,NaN,NaN,NaN,NaN,NaN
1,Accomack County,Virginia,2018,4973949.0,0.0,NaN,NaN,NaN,NaN,NaN
2,Accomack County,Virginia,2019,5397224.0,0.0,NaN,NaN,NaN,NaN,NaN
3,Ada County,Idaho,2017,1560323.0,10000.0,Ada County,Central,Idaho,14260.0,Metropolitan Statistical Area
4,Ada County,Idaho,2018,3254149.0,225000.0,Ada County,Central,Idaho,14260.0,Metropolitan Statistical Area


In [8]:
# add PDEN10 Mapper 

grants_per_county_cbsa['PDEN10'] = np.where(grants_per_county_cbsa['metropolitanmicropolitanstatis'] == 'Metropolitan Statistical Area',1,
                                np.where(grants_per_county_cbsa['metropolitanmicropolitanstatis'] == 'Micropolitan Statistical Area',
                                2,3))

In [9]:
grants_per_county_cbsa.to_csv("./Data/grants_per_county_cbsa.csv",index=False)

# # @Steph, the grants_per_county_cbsa is the df we want to visualize

In [11]:
grants_per_county_cbsa = grants_per_county.merge(cbsa_data, how='inner', left_on=['Complete County Name', 'State Name'], right_on=['countycountyequivalent', 'statename'])


grants_per_county_cbsa['Outlying'] = grants_per_county_cbsa['centraloutlyingcounty'].apply(lambda x: 1 if x =='Outlying' else 0)

grants_per_county_cbsa.tail(10)

,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,Outlying
701,Yamhill County,Oregon,1323089.0,0.0,Yamhill County,Outlying,Oregon,38900.0,1
702,Yates County,New York,381447.0,0.0,Yates County,Outlying,New York,40380.0,1
703,Yavapai County,Arizona,4230713.0,0.0,Yavapai County,Central,Arizona,39140.0,0
704,Yellowstone County,Montana,9541555.0,0.0,Yellowstone County,Central,Montana,13740.0,0
705,Yolo County,California,16361489.0,699965.0,Yolo County,Outlying,California,40900.0,1
706,York County,Maine,2541013.0,0.0,York County,Central,Maine,38860.0,0
707,York County,Pennsylvania,4298419.0,0.0,York County,Central,Pennsylvania,49620.0,0
708,York County,South Carolina,4883212.0,0.0,York County,Outlying,South Carolina,16740.0,1
709,Yuba County,California,2443802.0,0.0,Yuba County,Central,California,49700.0,0
710,Yuma County,Arizona,6683502.0,0.0,Yuma County,Central,Arizona,49740.0,0


In [12]:
grants_per_county_cbsa[["Outlying", "Total Active Grant Financial Assistance", 'Mental Health Assistance']].corr()

,Outlying,Total Active Grant Financial Assistance,Mental Health Assistance
Outlying,1.000000,-0.084758,-0.041312
Total Active Grant Financial Assistance,-0.084758,1.000000,0.302384
Mental Health Assistance,-0.041312,0.302384,1.000000


In [13]:
outlying = grants_per_county_cbsa[grants_per_county_cbsa['Outlying'] == 1]

central = grants_per_county_cbsa[grants_per_county_cbsa['Outlying'] == 0]

outlying[['Total Active Grant Financial Assistance','Mental Health Assistance']].describe()

,Total Active Grant Financial Assistance,Mental Health Assistance
count,8.100000e+01,81.000000
mean,3.596739e+06,27597.308642
std,4.244325e+06,135599.289789
min,3.881000e+03,0.000000
25%,7.123730e+05,0.000000
50%,2.221400e+06,0.000000
75%,5.076103e+06,0.000000
max,2.126549e+07,725000.000000


In [14]:
central[['Total Active Grant Financial Assistance','Mental Health Assistance']].describe()

,Total Active Grant Financial Assistance,Mental Health Assistance
count,6.300000e+02,6.300000e+02
mean,1.316710e+07,5.687492e+04
std,3.797416e+07,2.342490e+05
min,4.360000e+03,0.000000e+00
25%,1.205134e+06,0.000000e+00
50%,4.128082e+06,0.000000e+00
75%,9.863407e+06,0.000000e+00
max,4.436216e+08,3.254011e+06


In [15]:
grants_per_county_cbsa.shape

(711, 9)

In [17]:
grants_per_county_cbsa.to_csv('./Data/grants_data_cleaned.csv',index=False)